Imports and Utility


In [15]:
pip install -qU numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [16]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase, cosine_similarity
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Documents

## Loading Source Documents

In [17]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [18]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


## Splitting Text Into Chunks

In [19]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

In [20]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

# Task 3: Embeddings and Vectors

In [21]:
from aimakerspace.openai_utils.embedding import EmbeddingModel
embedding_model = EmbeddingModel()
puppy_sentence = "I love puppies!"
dog_sentence = "I love dogs!"
puppy_vector = embedding_model.get_embedding(puppy_sentence)
dog_vector = embedding_model.get_embedding(dog_sentence)
cosine_similarity(puppy_vector, dog_vector)

0.8340464507681294

In [22]:
puppy_sentence = "I love puppies!"
cat_sentence = "I dislike cats!"
puppy_vector = embedding_model.get_embedding(puppy_sentence)
cat_vector = embedding_model.get_embedding(cat_sentence)

cosine_similarity(puppy_vector, cat_vector)

0.3724333322983299

# Vector Database

## Question #1:
The default embedding dimension of text-embedding-3-small is 1536, as noted above.

1. Is there any way to modify this dimension? Yes pass `dimensions` parameter with the required value to modify default embedding dimension
2. What technique does OpenAI use to achieve this? OpenAI does it through `Matryoshka Representation Learning`

In [23]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

## Question #2:
What are the benefits of using an async approach to collecting our embeddings?

Benefits of using an async approach is performance. Performance can be improved for a code that is IO bound, when the system is waiting for a response for one request, it can process something else in a meanwhile (prepare and send all other request in our case). 

In [ ]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

# ChatOpenAI
## Question #3:
When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

# Creating and Prompting OpenAI's gpt-3.5-turbo!

In [ ]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]
response = chat_openai.run(messages)

In [ ]:
print(response)

There is no one "best" way to write a loop as it depends on the specific requirements of the task you are trying to accomplish. However, in general, it's good practice to keep your loops clear, concise, and readable. 

Here are some tips for writing effective loops:
1. Choose the appropriate loop type (for loop, while loop) based on the conditions of the task.
2. Use meaningful variable names to make your code more readable.
3. Consider using list comprehensions or generator expressions for more concise and elegant loop constructs.
4. Avoid nesting loops too deeply to maintain code clarity.
5. Make sure to update loop variables correctly and exit the loop when the condition is met.

Overall, the key is to write loops that are easy to understand, maintain, and debug. Remember to always test your loops thoroughly to ensure they are functioning as expected.


# Task 5: Retrieval Augmented Generation

In [ ]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_user_prompt, formatted_system_prompt]), "context" : context_list}

## Question #4:
What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

In [ ]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [ ]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")


{'response': 'The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO, who previously excelled in a particular function (such as product management, sales, or marketing), hires a weak executive to lead that same function in order to retain control and continue to be seen as the primary decision-maker. This behavior is illustrated by the example of Michael Eisner, the former CEO of Disney, who struggled with turning around ABC after acquiring it due to his focus on his previous expertise as a TV network executive rather than effectively managing the new acquisition. This problem highlights the tendency of some executives to hire weaker individuals in key roles to maintain their own dominance and control.',
 'context': [('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who

# Activity #1:
Enhance your RAG application in some way!
- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

# Visibility Tooling

In [ ]:
pip install -qU wandb

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import os
wandb_key = getpass.getpass("Weights and Biases API Key: ")
os.environ["WANDB_API_KEY"] = wandb_key

In [ ]:
import wandb

wandb.init(project="Visibility Example - AIE3")

In [ ]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace

class RetrievalAugmentedGenerationPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)


        start_time = datetime.datetime.now().timestamp() * 1000

        try:
            openai_response = self.llm.run([formatted_system_prompt, formatted_user_prompt], text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""

        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")

        return {"response" : response_text if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message }


NameError: name 'ChatOpenAI' is not defined